# [Module 4.2.2] inference on custem tf serving image

이 노트북은 아래와 같은 작업을 진행 합니다.
- Custom Inference Docker Image를 사용하여, SageMaker Model을 생성
- 위의 SageMaker Model를 통해서 Deploy 하여 Endpoint를 생성 합니다.
- Endpoint에 Predictor를 생성 합니다.
- 추론을 하여 상위 5개의 높은 스코어를 가진 것을 이모팀콘으로 변경하여 5개 추천 합니다.

---
이 노트북은 약 10 분 정도 시간이 소요 됩니다.

In [1]:
import boto3
import sagemaker
import pandas as pd
import os

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

In [2]:
%store -r 

In [3]:
# training_job_name = 'bert2tweet-2020-07-08-07-58-27-895'
print(training_job_name)

bert2tweet-2020-08-19-00-05-42-310


In [4]:
inference_image = ecr_infer_custom_image_tf_serving_20_cpu
print("inference_image: ", inference_image)

inference_image:  343441690612.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-tensorflow-serving:2.0.0-cpu


## Custome TFS Docker Image 및 Inference code 로 모델 생성

In [5]:
from sagemaker.tensorflow.serving import Model

model = Model(model_data='s3://{}/{}/output/model.tar.gz'.format(bucket, training_job_name),
              role=role,
              entry_point='inference.py',
              image = inference_image
             ) 

In [6]:
print(model.image)
print(model.serving_image_uri)
print(model.model_data)
print(model.name)
print(model.FRAMEWORK_NAME)


343441690612.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-tensorflow-serving:2.0.0-cpu
<bound method Model.serving_image_uri of <sagemaker.tensorflow.serving.Model object at 0x7f6b7de188d0>>
s3://sagemaker-ap-northeast-2-343441690612/bert2tweet-2020-08-19-00-05-42-310/output/model.tar.gz
None
tensorflow-serving


## 엔드포인트 생성

In [7]:
%%time

instance_type='ml.m4.xlarge'
deployed_model = model.deploy(initial_instance_count = 1,
                             instance_type = instance_type,
                             wait=True)


-------------!CPU times: user 39.1 s, sys: 6.12 s, total: 45.2 s
Wall time: 7min 16s


## Predictor Creation on the Endpoint

In [8]:
# tweet_bert_endpoint_name = 'train_text, train_label, test_text, test_label = tweet_data.split_train_test_data(texts, labels, 0.9)
tweet_bert_endpoint_name = deployed_model.endpoint
print(tweet_bert_endpoint_name)

sagemaker-tensorflow-serving-2020-08-19-00-52-49-431


In [9]:
import json
from sagemaker.tensorflow.serving import Predictor

predictor = Predictor(endpoint_name = tweet_bert_endpoint_name,
                      sagemaker_session = sess,
                      content_type = 'application/json',
                     )

## Inference 실행

In [22]:
from TweetUtil import TweetUtil

tweet_util = TweetUtil()
tweet_util.load_emoji_data('emoji_to_idx.pickle')
emoji = tweet_util.get_emo_class_label(3)
print(emoji)

emoji_to_idx is loaded
😂


In [36]:
test_file_path = 'data/test/tweet_file_test.csv'
test_df = pd.read_csv(test_file_path)
test_file_path = 'data/test/tweet_file_test.csv'
test_df = pd.read_csv(test_file_path)
sample_df = test_df.sample(10)
sample_df

,TWEET,LABEL
8561,me qnd my buddy,4
5636,apink prepared a coffee truck to the pandas w...,0
2965,brix a grown man now is he looking for colleg...,7
9129,yep got my ipad charging ready to rock,0
6946,i wasted two years at psg in ligue 1 it's lik...,2
7189,guys go vote for him he has a chance to win a...,0
4286,nouvellevideo mes produits high tech favoris rt,2
3332,i sleeps on my sofa every night,6
1708,good morning i love you,1
3889,last night part 2,5


In [37]:
def show_top_N_label(score_list, topN):

    import numpy as np

    top_n_idx = np.argsort(score_list)[-topN:]
    top_n_values = [score_list[i] for i in top_n_idx]
    
    top_n_idx_list = top_n_idx.tolist()
    top_n_idx_list.reverse()
    top_n_values = [score_list[i] for i in top_n_idx_list]    
    
    return top_n_idx_list



## Top 5 이모티콘 추천

In [39]:
import tensorflow as tf
import json

columns = ['TWEET', 'LABEL']
topN = 5
for tweet, label in zip(sample_df.TWEET.values, sample_df.LABEL.values):
    # print("label: {}, tweet: {}".format(label, tweet))
    
    reviews = [tweet]
    
#     print("reviews: \n", reviews)



    predicted_classes = predictor.predict(reviews)[0]
#    predicted_classes = predictor.predict(reviews)    
    predicted_classes = show_top_N_label(predicted_classes, topN)

    print('-------------------------------------------')        
    print('tweet: {} \nGround_truth- {}:{}\n '.format(
        tweet,
        label, 
        tweet_util.get_emo_class_label(label))
         )    
    

    print('Prediction: {},{},{},{},{},{},{},{},{},{} \n '.format(
        predicted_classes[0], 
        tweet_util.get_emo_class_label(predicted_classes[0]),
        predicted_classes[1], 
        tweet_util.get_emo_class_label(predicted_classes[1]),
        predicted_classes[2], 
        tweet_util.get_emo_class_label(predicted_classes[2]),                                       
        predicted_classes[3], 
        tweet_util.get_emo_class_label(predicted_classes[3]),                                      
        predicted_classes[4], 
        tweet_util.get_emo_class_label(predicted_classes[4]),                                      
        
        ))    
        




-------------------------------------------
tweet: me qnd my buddy  
Ground_truth- 4:😊
 
Prediction: 3,😂,1,💕,0,❤,7,😭,5,😍 
 
-------------------------------------------
tweet:  apink prepared a coffee truck to the pandas who went to support them during the pre recording how sweet cr sweetyb 
Ground_truth- 0:❤
 
Prediction: 5,😍,3,😂,7,😭,4,😊,8,🙄 
 
-------------------------------------------
tweet:  brix a grown man now is he looking for colleges with d1 swimming  
Ground_truth- 7:😭
 
Prediction: 9,🤔,3,😂,6,😩,8,🙄,7,😭 
 
-------------------------------------------
tweet:  yep got my ipad charging ready to rock  
Ground_truth- 0:❤
 
Prediction: 2,🔥,5,😍,8,🙄,6,😩,7,😭 
 
-------------------------------------------
tweet:  i wasted two years at psg in ligue 1 it's like playing fifa on the beginner level david luiz 
Ground_truth- 2:🔥
 
Prediction: 2,🔥,7,😭,3,😂,6,😩,5,😍 
 
-------------------------------------------
tweet:  guys go vote for him he has a chance to win a scholarship to his dream school 

In [32]:
%store tweet_bert_endpoint_name

Stored 'tweet_bert_endpoint_name' (str)
